# core lib

In [1]:
import numpy as np
import pandas as pd
import io
import calendar
from datetime import datetime
from datetime import timedelta
from operator import attrgetter

from sklearn.metrics import precision_score, f1_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

# Log lib
import logging
from ast import literal_eval
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

ModuleNotFoundError: No module named 'sklearn'

# Plot lib 

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as st

# Standard plotly imports
# import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
# cufflinks.go_offline()
init_notebook_mode(connected=True)

ModuleNotFoundError: No module named 'cufflinks'

# Path to data mapping

In [ ]:
input_common_directory = '/data/apps/jupyter_notebook/EDA/common_eda/input_common'

In [3]:
path_north_province = input_common_directory + "/3MienVN/mienbac.csv";
path_central_province = input_common_directory + "/3MienVN/mientrung.csv";
path_south_province = input_common_directory + "/3MienVN/miennam.csv";

path_miennuibacbo = input_common_directory +  "/7vungkinhtevn/trungduvamiennuibacbo.csv"
path_dongbangsonghong =input_common_directory +  "/7vungkinhtevn/dongbangsonghong.csv"
path_bactrungbo =input_common_directory +  "/7vungkinhtevn/bactrungbo.csv"
path_namtrungbo =input_common_directory +  "/7vungkinhtevn/namtrungbo.csv"
path_taynguyen =input_common_directory +  "/7vungkinhtevn/taynguyen.csv"
path_dongnambo =input_common_directory +  "/7vungkinhtevn/dongnambo.csv"
path_dongbangsongcuulong =input_common_directory +  "/7vungkinhtevn/dongbangsongcuulong.csv"

path_mapping_telco = input_common_directory + "/dausodt.csv"
path_mapping_career = input_common_directory + "/career_mapping.csv";
path_location_mapping = input_common_directory + "/outdata_norm_id_4_location_mapping.csv";

NameError: name 'input_common_directory' is not defined

# auto eda lib (enable if need)

In [4]:
# from dataprep.eda import plot, plot_correlation, plot_missing
# from pandas_profiling import ProfileReport
# import sweetviz as sv

# Config UI

In [4]:
# In Jupyter Notebooks, one clean way of solving this problem is using markdown:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
pd.options.display.float_format = "{:.4f}".format
pd.options.display.max_rows = 500
pd.set_option('max_columns', 50)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [5]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# Describe Data

In [9]:
def missing_explore_lite(input_dataframe):
    """
        Show simple overral missing of dataframe columns
        Args:
            input_dataframe: pandas dataframe
        Returns:
            Table simple missing explore by columns (combine three columns: count_missing,count_total, percentile_missing)
    """
    printmd("**Missing Explore**")
    naCount = input_dataframe.isnull().sum()
    total = len(input_dataframe)
    naPercent = (input_dataframe.isnull().sum()/len(input_dataframe)*100).round(2).map(lambda n: '{0:.1f} %'.format(n))
    return pd.DataFrame({'count_missing': naCount, 'count_total': len(input_dataframe),'percentile_missing':naPercent})

def missing_explore_full(input_dataframe):
    """
        Show full overral missing of dataframe columns
        Args:
            input_dataframe: pandas dataframe
        Returns:
            Table full missing explore by columns (columns: count_total, count_unique, count_duplicate, count_zero, 
                                            percentile_zero, count_missing, percentile_missing, hit_rate)
    """
    printmd("**Overral Stats**")
    total = len(input_dataframe)
    naCount = input_dataframe.isnull().sum()
    zeroCount = len(input_dataframe) - input_dataframe.fillna(1).astype(bool).sum()
    zeroPercent = (zeroCount/len(input_dataframe)*100).round(2).map(lambda n: '{0:.2f} %'.format(n))
    naPercent = (input_dataframe.isnull().sum()/len(input_dataframe)*100).round(2).map(lambda n: '{0:.2f} %'.format(n))
    uniqCount = input_dataframe.nunique()
    dupCount = []
    for column_name in input_dataframe.columns:
        dupCount.append(input_dataframe.duplicated(subset=column_name, keep='first').sum())
    hitRate = (input_dataframe.notnull().sum()/len(input_dataframe)*100).round(2).map(lambda n: '{0:.2f} %'.format(n))
    return pd.DataFrame({'count_total': total, 'count_unique': uniqCount, 'count_duplicate': dupCount, 'count_zero':zeroCount,'percentile_zero':zeroPercent, 'count_missing': naCount,'percentile_missing':naPercent, 'hit_rate':hitRate})

def hit_rate_explore(df, column):
    """
        Calculate percentage null and not null value of column in data frame
        Args:
            df: input dataframe
            column: name of column need calculate hit rate
        Returns:
            hitrate table of column
    """
    na_count = df[column].isnull().sum()
    not_na_count = df[column].notnull().sum()
    total = len(df)
    na_percentage = '{0:.1f} %'.format(na_count/total*100)
    not_na_percentage = '{0:.1f} %'.format(not_na_count/total*100)
    return pd.DataFrame({'Category': ['Have ' + column, 'Does not have ' + column],
                        'Count': [not_na_count, na_count],
                        'Total': [total, total],
                        'Percent': [not_na_percentage, na_percentage]})

#Describe Data:
def describe_numeric_data(column, round=2):
    """
        Show describe numeric data
        Args:
            column: pandas serie datatype is numeric
            round: round format number (default = 2)
        Returns:
            Table describe data (total, std, pecentiles,...)
    """
    return column.describe(percentiles = [i*0.05 for i in range (20)] +[0.01] +[0.99]).round(round)

def describe_category_data(column):
    """
        Show describe category data
        Args:
            column: pandas serie datatype is category
        Returns:
            Table describe data (count values, percentiles)
    """
    c = column.value_counts(dropna=False)
    p = column.value_counts(dropna=False, normalize=True).map(lambda n: '{0:.2f} %'.format(n*100))
    return pd.concat([c,p], axis=1, keys=['counts', 'percentiles(%)'])

def describe_category_data_without_format(column):
    """
        Show describe category data
        Args:
            column: pandas serie datatype is category
        Returns:
            Table describe data (count values, percentiles)
    """
    c = column.value_counts(dropna=False)
    p = column.value_counts(dropna=False, normalize=True)
    return pd.concat([c,p], axis=1, keys=['counts', 'percentiles'])

def descibe_2D_category_data(df_input, column1, column2):
    """
        Show describe table 2 categories data
        Args:
            df_input: dataframe contain 2 categories column
            column1: category column 1
            column1: category column 2
        Returns:
            Table describe data (count values, percentiles)
    """
    df_count = df_input.groupby([column1, column2])[column2].size().unstack(fill_value=0)
    values = df_input[column2].dropna().unique()
    df_percentile = pd.DataFrame()
    for value in values:
        df_percentile['percentile_' + str(value)] =  df_count[value]/df_count.sum(axis=1)
        df_percentile['percentile_' + str(value)] = df_percentile['percentile_' + str(value)].apply(lambda x: "{0:.2f} %".format(x*100))
    return df_count.join(df_percentile)

## Print False Case

In [7]:
def get_false_case(labels_list, df, pred_col_name, label_col_name):
    """
    get the list of dataframe of false case
    :return: list of dataframe
    """
    false_df_list = []
    for label in labels_list:
        false_df = df[(df[pred_col_name] != df[label_col_name])&(df[label_col_name] == label)]
        false_df_list.append(false_df)
    return false_df_list

def show_false_case(labels_list, df, pred_col_name, label_col_name, topn=5):
    """
    show top false case of pandas serie
    :return: list of false case
    """
    dflist = get_false_case(labels_list, df, pred_col_name, label_col_name)
    for dfitem, label in zip(dflist, labels_list):
        if (label!= "nan") and (dfitem.shape[0] > 0):
            print(color.BOLD + "False of label '" + str(label) + "'" + color.END)
            display(dfitem[dfitem.columns.difference(['uid'])].head(topn))
            print("===========================================================================================")
            print()
  

# Draw Chart

## Private function

In [ ]:
# Draw Chart
def _draw_bar_plotly_1D_with_orders(df_input, xTitle, labels = []):
    '''
        Private function draw bar chart for describe category column
        Args: 
            df_input: dataframe groupby column need describe
            xTitle: name of column need show in xTitle
            labels: list category orders when plot chart 
        Returns:
            Show bar chart for category column data
    '''
    n_cates = len(df_input)

    width = 300 if (n_cates <= 2) else n_cates * max(50, 140 * np.power(0.92, n_cates / 2))
    fig = px.bar(df_input,
                 x=df_input.index,
                 y='counts',
                 text=df_input['percentiles(%)'],
                 width=width,
                 height=500,
                 category_orders={column: labels})
    fig.update_layout(transition_duration=500)
    fig.update_layout(
        title=title + " Chart",
        xaxis_title=title,
        yaxis_title="Counts"
    )
    fig.update_layout(transition_duration=500)
    fig.show()

In [ ]:
def draw_hist_plotly_1D(input_data, xTitle, title):
    '''
        Plotly draw histogram chart for continuous data
        Args: 
            input_data: pandas serie continuous column need plot chart (example: df[column_name])
            xTitle: title of xAxis (maybe column name)
            title: title of Chart
        Returns:
            Show histogram chart for data
    '''
    input_data.astype(str).iplot(kind='hist', xTitle=xTitle, yTitle='count', title=title)
    
def draw_bar_plotly_1D(input_data):
    '''
        Plotly draw bar chart for category data
        Args: 
            input_data: pandas serie category column need plot chart (example: df[column_name])
        Returns:
            Show histogram chart for data
    '''
    fig = px.bar(input_data.value_counts())
    fig.show()

def draw_pie_plt_1D(input_data):
    
    labels = input_data.value_counts(dropna=False).index
    values = input_data.value_counts(dropna=False).values
    fig1, ax1 = plt.subplots()
    ax1.pie(values, labels=labels, autopct='%1.1f%%',
            shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

def draw_pie_plotly_1D(input_data):
    labels = input_data.value_counts(dropna=False).index
    values = input_data.value_counts(dropna=False).values
    fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
    fig.show()

## Public function

### 1D

In [ ]:
def describe_numeric_data_with_chart_hist(df_input, column_name, round=2):
    """
        Describe data and plot histogram of numeric data
        Arguments:
            df_input: dataframe has column
            column_name: name of column need describe
            round: number of round data
    """
    print(color.BOLD + "Describe data of " + column_name + ":" + color.END)
    display(describe_numeric_data(df_input[column_name], round))
    print(color.BOLD + "Historam chart of " + column_name + ":" + color.END)
    if len(df_input) > 100000:
        draw_hist_plotly_1D_matplotlib(df_input[column_name], column_name, "Chart count of " + column_name)
    else:
        draw_hist_plotly_1D(df_input[column_name], column_name, "Chart count of " + column_name)
    
def describe_category_data_with_chart_bar(df_input, column_name, labels = []):
    """
        Describe data and plot bar chart of category data
        Arguments:
            df_input: dataframe has column
            column_name: name of column need describe
            labels: list sort labels when plot (default empty array will sort by count value decreasing)
    """
    column = df_input[column_name]
    dfg = describe_category_data(column)
    print(color.BOLD + "Describe data of " + column_name + ":" + color.END)
    display(dfg)
    print(color.BOLD + "Historam chart of " + column_name + ":" + color.END)
    if len(labels) == 0:
        if not np.issubdtype(df_input[column_name].dtype, np.number) and not column_name == 'age_group':
            labels = df_input[column_name].value_counts().index
        else:
            labels = df_input[column_name].dropna().unique()
            labels.sort()
#     if len(df_input) > 100000: # Plotly xài groupby rồi nên rất nhẹ
#         draw_bar_matplotlib_1D_with_orders(df_input, column_name, labels)
#     else:
    _draw_bar_plotly_1D_with_orders(dfg, column_name, "Distributed of " + column_name, labels)
    
def describe_category_data_with_chart_pie(column, column_name):
    """
        Describe data and plot bar chart of category data
        Arguments:
            df_input: dataframe has column
            column_name: name of column need describe
            labels: list sort labels when plot (default empty array)
    """
    print(color.BOLD + "Describe data of " + column_name + ":" + color.END)
    display(describe_category_data(column))
    print(color.BOLD + column_name + " distribution:" + color.END)
    draw_pie_plotly_1D(column)

def describe_category_with_draw_hist_plotly_2D(df_input, column1, column2):
    """
        Describe 2D data and plot histogram chart of category data
        Arguments:
            df_input: dataframe has column
            column1: name of category column 1
            column2: name of category column 2
    """
    print(color.BOLD + "Describe data of " + column1 + " and " + column2 +":" + color.END)
    display(descibe_2D_category_data(df_input, column1, column2))
    print(color.BOLD + "Data distribution:" + color.END)
    draw_hist_plotly_2D(df_input, column1, column2)
    
def draw_line_time_series_chart_plotly_1D(df_input, column):
    """
        Plot time series data to chart line
        Arguments:
            df_input: data frame has column need plot
            column: name of time series column
    """
    column_datetime_format = df_input[column].dt.strftime('%Y/%m')
    time_series = pd.DataFrame(column_datetime_format.value_counts().reset_index())
    time_series.columns = ['date', 'count']
    time_series = time_series.sort_values('date', ascending=True)
    fig = px.line(time_series, x="date", y="count", title='Count by ' + column)
    fig.show()

### 2D

In [8]:


def draw_line_ploty_mean_2D(df_input, column1, column2):
    df_input = df_input.groupby([column1], as_index=False).mean()
    fig = px.line(df_input, x=column1, y=column2, title=''.join([column1,' + ',column2]))
    fig.show()
    
def draw_line_ploty_2D_with_filter_na(df_input, column1, column2):
    df_input = df_input[(df_input[column1].notnull()) & (df_input[column1] > 0) & (df_input[column2].notnull())][[column1, column2]]
    draw_line_ploty_2D(df_input, column1, column2)
    
def draw_hist_plotly_2D(df_input, x_axis, color):
    fig = px.histogram(df_input, x=x_axis, color=color,
                   barmode='relative',
                   hover_data=df_input.columns)
    fig.show()
    
def draw_bar_plotly_2D(df_input, x_axis, color):
    fig = px.bar(df_input, x=x_axis, color=color,
                   barmode='relative',
                   hover_data=df_input.columns)
    fig.show()
    
def draw_cate_and_cate_chart_nested_bar_count_plotly_2D(df_input, columnX, columnY):
    inputdf = df_input[(df_input[columnX].notnull()) & (df_input[columnY].notnull())]
    fig = px.histogram(inputdf, x=columnX, y=columnY, color=columnY, histfunc="count", barmode="group")
    fig.show()

def draw_cate_and_cate_chart_stacked_bar_percent_plotly_2D(df_input, columnX, columnY):
    inputdf = df_input[(df_input[columnX].notnull()) & (df_input[columnY].notnull())]
    fig = px.histogram(inputdf, x=columnX, y=columnY, color=columnY, histfunc="count", barmode="relative", barnorm='percent')
    fig.show()
    
def describe_2D_fullflow(df_input, column1, column2, labels_list, figsize):
#     df_without_invalid_uid = df_input[df_input.uid.notnull()]
#     print(plot_confusion_matrix(df_input[column1].astype(str), df_input[column2].astype(str), figsize=figsize, title="Confusion Matrix With Full Data"))
#     print(plot_confusion_matrix(df_without_invalid_uid[column1].astype(str), df_without_invalid_uid[column2].astype(str), figsize=figsize, title="Confusion Matrix Without Invalid Uid Data"))
    df_input_notnull=df_input[df_input[column1].notnull() & df_input[column2].notnull()]
    print(plot_confusion_matrix(df_input_notnull[column1].astype(str), df_input_notnull[column2].astype(str), figsize=figsize, title="Confusion Matrix Without Nan Data"))
    # show false case
    unique_labels_list = df_input[df_input[column1].notnull()][column1].unique()
    show_false_case(unique_labels_list, df_input, column1, column2, topn=5)

### Matplotlib

In [7]:
def draw_hist_plotly_1D_matplotlib(x, xTitle, title):
    '''Plot Historgram using matplotlib'''
    plt.figure(figsize=(10,8))
    plt.hist(x, bins=30, label=title)
    plt.ylabel('Count')
    plt.xlabel(xTitle)
    plt.title("Histogram Count")
    plt.show()
    
    plt.figure(figsize=(10,8))
    plt.hist(x, bins=30, density=True)
    sns.kdeplot(x);
    plt.ylabel('Probability')
    plt.xlabel(xTitle)
    plt.title("Histogram Probability")
    plt.show()
    
def draw_bar_matplotlib_1D_with_orders(df_input, column_name, order_labels):
    plt.figure(figsize=(10,8))
    sns.set(style='darkgrid')
    ax = sns.countplot(x=column_name, data=df_input, order = order_labels)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
    plt.tight_layout()
    plt.show()
    
def draw_box_seaborn_2D(df_input, x, y):
    plt.figure(figsize=(20,10))
    ax = sns.boxplot(data=df_input, x=x, y=y)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
    plt.tight_layout()
    plt.show()
    
def draw_cate_and_cate_chart_nested_bar_count_seaborn_2D(df_input, x, hue):
    plt.figure(figsize=(10,8))
    sns.set(style='darkgrid')
    ax = sns.countplot(x=x,hue=hue,data=df_input)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
    plt.tight_layout()
    plt.show()

## function util

In [9]:
def add_months(sourcedate, n_months):
    '''
        Add n_months month to sourcedate
        Args:
            sourcedate (datetime): input date need add months
            n_months: number of months need add
        Returns: 
            datetime result after add n_months
    '''
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year,month)[1])
    return datetime(year, month, day)

In [10]:
def remove_unicode(data):
    """
    Remove unicode characters from string using lib unidecode
        Args:
            data: Input string
        
        Returns:
            decode_data: string removed unicode characters
            
    """
    return unidecode.unidecode(data)


# Format Data

In [13]:
def format_gender(gender):
    '''
        Format gender 
    '''
    if gender == 1:
        return "Male"
    elif gender == 2:
        return "Female"
    return np.nan

def parentalstatus_mapping(parentalstatus):
    if parentalstatus == 'NOT_A_PARENT':
        return 'Not_a_parent'
    elif parentalstatus == 'PARENT':
        return 'Parent'
    return np.nan

def maritalstatus_mapping(status):
    if status == 0:
        return 'Unknown'
    elif status == 1:
        return 'Married'
    return np.nan

#Name
def getNameFromFullName(fullname):
    arr_name = fullname.split()
    return arr_name[len(arr_name) - 1].capitalize()

#Age
def getAgeFromDOB(dob):
    
    arr_dob = dob.split("/")
    now = datetime.now()
    return now.year - int(arr_dob[len(arr_dob) - 1])

def delta_age(age1, age2):
    """
        Calculate delta between two age data
        Arguments: 
            age1: predited age
            age2: actual age
        Returns:
            delta_age
    """
    if(abs(age1 - age2) > 5):
        return '>5'
    return '=' + str(abs(age1 - age2))

def format_limit_data_with_top_number_value(df_input, column_name, ntop=10, other_name="Others"):
    """
        Format category data to top data and others value
        Arguments: 
            df_input: dataframe input
            column_name: name of columns need to format
            ntop: number of top value 
            other_name: name of other value
    """
    df_counts = df_input[column_name].value_counts().sort_values(ascending=False).rename_axis('name').reset_index(name='count')
    serie_column = df_input[column_name].copy()
    list_top_data = df_counts.head(n=ntop)['name'].tolist()
    serie_column.loc[~serie_column.isin(list_top_data) & serie_column.notnull()] = other_name
    df_input[column_name + '_limit'] = serie_column

## Remove invalid format data row

In [14]:
def func_format_some_feature_internal(df):
    '''
        Format some feature internal if feature is in dataframe 
        List features support ('agev2_predicted', 'genderv2_predicted', 'maritalstatus_predicted', 
                                'parentalstatus_predicted', 'home_predicted', 'work_predicted', 'livingprovince_predicted')
        Args:
            df: dataframe input need format
        Returns:
            dataframe with list features was formatted
    '''
    
    if 'agev2_predicted' in df.columns:
        df['agev2_predicted'] = df[df['agev2_predicted'].notnull()]['agev2_predicted'].astype(int)
    if 'genderv2_predicted' in df.columns:
        df['genderv2_predicted'] = df[df['genderv2_predicted'].notnull()]['genderv2_predicted'].astype(int).apply(format_gender)
    if 'maritalstatus_predicted' in df.columns:
        df['maritalstatus_predicted'] = df['maritalstatus_predicted'].apply(maritalstatus_mapping)
    if 'parentalstatus_predicted' in df.columns:
        df['parentalstatus_predicted'] = df['parentalstatus_predicted'].apply(parentalstatus_mapping)
    if 'home_predicted' in df.columns:
        df['home_predicted'] = df['home_predicted'].apply(lambda x: f'{x:.0f}').astype(str)
    if 'work_predicted' in df.columns:
        df['work_predicted'] = df['work_predicted'].apply(lambda x: f'{x:.0f}').astype(str)
    if 'livingprovince_predicted' in df.columns:
        df['livingprovince_predicted'] = df['livingprovince_predicted'].apply(lambda pr: pr.replace('Thành phố ', '').replace('Tỉnh ', '') if isinstance(pr, str) else np.nan)
    return df

## Data Dic Career

In [17]:

# Clean Input Data
with open(path_mapping_career) as f:
    fileobject = io.StringIO(f.read().replace(",'","").replace("'",""))
# Read Map Career 
dic_career = pd.read_csv(fileobject, sep='\t', header=None, index_col=0, squeeze=True).to_dict()
# Extract out keys and values
k_car = np.array(list(dic_career.keys()))
v_car = np.array(list(dic_career.values()))

## Data Dic Province

In [18]:
# Read Map Career 
df_location = pd.read_csv(path_location_mapping, sep='\t', header=None, squeeze=True)
df_location[6] = df_location[6].astype(str).apply(lambda loc: ''.join(['241', loc.zfill(6)]))
dic_location = df_location.set_index(6)[7].to_dict()

In [19]:
#Convert Data
def replace_with_dict_location(province_id):
    try:
        if province_id:
            return dic_location[province_id]
        else:
            return np.nan
    except:
        return np.nan

## Format Category with Top data

In [20]:
def func_format_mapping_location(df):
    list_mapping_location = ['home_predicted', 'work_predicted']
    for column in list_mapping_location:
        if column in df.columns:
            df[column + '_province'] = df[df[column].notnull()][column].apply(replace_with_dict_location)
    return df

In [4]:
def func_format_top_data(df, ntop = 10):
    """
        Format all category features if exist in dataframe to list top data and others 
        ('occupationFirst_predicted','livingprovince_predicted', 'home_predicted_province', 'work_predicted_province', 'hometown_predicted')
        Arguments: 
            df: data frame need format
            ntop: number of top data (others will be assign for other datas)
        Returns:
            df: data frame contain format data
    """
    list_get_top_data = ['occupationFirst_predicted','livingprovince_predicted', 'home_predicted_province', 'work_predicted_province', 'hometown_predicted']

    for column in list_get_top_data:
        if column in df.columns:
            format_limit_data_with_top_number_value(df, column, ntop, "Others")
    return df 


# function mapping-convert data

## Private function

In [ ]:
def _get_phone_start_of_raw_phone(phone):
    try:
        if phone:
            phone = str(phone)
            if len(phone) < 9 or len(phone) >=12:
                return ''
            if len(phone) >= 11 and phone[:2] == '84':
                phone = phone[2:]
            elif phone[:1] == '0':
                phone = phone[1:]
            return phone[:2]
    except:            
        return ''
    return ''

def _group_age_apply_func(age):
    if np.isnan(age) or age is None or age <= 0:
        return np.nan
    elif age <= 17:
        return "17 Trở xuống"
    elif age <= 24:
        return "18-24"
    elif age <= 34:
        return "25-34"
    elif age <= 44:
        return "35-44"
    elif age <= 59:
        return "45-59"
    return "60 Trở lên"
    

## Public function

In [ ]:
def group_age_mapping(df_input, age_column_name):
    '''
        Get group age from age column
        Arg:
            df_input: dataframe contain age column
            age_column_name: age column name in dataframe
        Return:
            pandas series group age combine list group("17 Trở xuống", "18-24", "25-34", "35-44", "45-59", "60 Trở lên")
    '''
    return df[age_column_name].apply(_group_age_apply_func)

def gender_mapping(gender):
    if gender == 1:
        return 'Nam'
    elif gender == 2:
        return 'Nữ'
    return np.nan

def telco_mapping(df_input, phone_column_name, type_phone = 'raw'):
    '''
        Get telco name from phone column
        Arg:
            df_input: dataframe contain phone column
            phone_column_name: phone column name in dataframe
            type_phone: 'raw' or 'noise'
        Return:
            pandas series telco name
    '''
    df_telco = pd.read_csv(path_mapping_telco, sep=',', header=None, squeeze=True)
    df_telco.columns= ["phone_start", "telco_name", "noise_phone_start"]
    df_input_merge = df_input[[phone_column_name]].dropna().drop_duplicates()
    if type_phone == 'noise':
        df_telco["phone_start_formatted"] = df_telco["noise_phone_start"].astype(str)
        df_input_merge["phone_start_formatted"] = df_input_merge[phone_column_name].astype(str).str[0:7]
    else:
        df_telco["phone_start_formatted"] = df_telco["phone_start"].astype(str)
        df_input_merge["phone_start_formatted"] = df_input[phone_column_name].apply(_get_phone_start_of_raw_phone)
    df_input_merge = pd.merge(left=df_input_merge, right=df_telco[["phone_start_formatted", "telco_name"]], how='left', on='phone_start_formatted')
    display(df_input_merge)
    return pd.merge(left=df_input, right=df_input_merge[[phone_column_name, "telco_name"]], how='left', on=phone_column_name)["telco_name"]

def format_province_by_3zone(serie_province): 
    df_north_province = pd.read_csv(path_north_province, sep='\t', header=None, squeeze=True)
    df_central_province = pd.read_csv(path_central_province, sep='\t', header=None, squeeze=True)
    df_south_province = pd.read_csv(path_south_province, sep='\t', header=None, squeeze=True)
    serie_province.loc[serie_province.isin(df_north_province.tolist())] = 'Các tỉnh miền Bắc'
    serie_province.loc[serie_province.isin(df_central_province.tolist())] = 'Các tỉnh miền Trung'
    serie_province.loc[serie_province.isin(df_south_province.tolist())] = 'Các tỉnh miền Nam'
    return serie_province

def format_province_by_5zone(serie_province): 
    whitelist = ["Hồ Chí Minh", "Hà Nội"]
    df_north_province = pd.read_csv(path_north_province, sep='\t', header=None, squeeze=True)
    df_central_province = pd.read_csv(path_central_province, sep='\t', header=None, squeeze=True)
    df_south_province = pd.read_csv(path_south_province, sep='\t', header=None, squeeze=True)
    serie_province.loc[(serie_province.isin(df_north_province.tolist())) & (~serie_province.isin(whitelist))] = 'Các tỉnh miền Bắc'
    serie_province.loc[(serie_province.isin(df_central_province.tolist())) & (~serie_province.isin(whitelist))] = 'Các tỉnh miền Trung'
    serie_province.loc[(serie_province.isin(df_south_province.tolist())) & (~serie_province.isin(whitelist))] = 'Các tỉnh miền Nam'
    return serie_province
    
def format_province_by_7zone(serie_province): 
    df_path_miennuibacbo = pd.read_csv(path_miennuibacbo, sep='\t', header=None, squeeze=True)
    df_path_dongbangsonghong = pd.read_csv(path_dongbangsonghong, sep='\t', header=None, squeeze=True)
    df_path_bactrungbo = pd.read_csv(path_bactrungbo, sep='\t', header=None, squeeze=True)
    df_path_namtrungbo = pd.read_csv(path_namtrungbo, sep='\t', header=None, squeeze=True)
    df_path_taynguyen = pd.read_csv(path_taynguyen, sep='\t', header=None, squeeze=True)
    df_path_dongnambo = pd.read_csv(path_dongnambo, sep='\t', header=None, squeeze=True)
    df_path_dongbangsongcuulong = pd.read_csv(path_dongbangsongcuulong, sep='\t', header=None, squeeze=True)
    serie_province.loc[serie_province.isin(df_path_miennuibacbo.tolist())] = 'Trung du và miền núi Bắc bộ'
    serie_province.loc[serie_province.isin(df_path_dongbangsonghong.tolist())] = 'Đồng bằng sông Hồng'
    serie_province.loc[serie_province.isin(df_path_bactrungbo.tolist())] = 'Bắc Trung bộ'
    serie_province.loc[serie_province.isin(df_path_namtrungbo.tolist())] = 'Nam Trung bộ'
    serie_province.loc[serie_province.isin(df_path_taynguyen.tolist())] = 'Tây Nguyên'
    serie_province.loc[serie_province.isin(df_path_dongnambo.tolist())] = 'Đông Nam bộ'
    serie_province.loc[serie_province.isin(df_path_dongbangsongcuulong.tolist())] = 'Đồng bằng sông Cửu Long'
    return serie_province